In [2]:
#!/usr/bin/python3

# Sample API client for StreetLight Data API
# This was tested with Python 3 on Ubuntu Linux 16.04.
# Refer to the StreetLight API documentation for more information.

import datetime as dt
import json
import requests
import sys
import time
import pandas as pd

In [3]:
dfZoneSets = pd.DataFrame([
    ['Recreation_Zones_CacheCo'        ,'StreetLight_TAZ_RecreationZones_CacheCo'        ,0,'Pre-Set OD'],
    ['Recreation_Zones_UDOT'           ,'StreetLight_TAZ_RecreationZones_UDOT'           ,0,'Pre-Set OD'],
    ['Recreation_Zones_WasthingtonCo'  ,'StreetLight_TAZ_RecreationZones_WashingtonCo'   ,0,'Pre-Set OD'],
    ['Recreation_Zones_WFRC'           ,'StreetLight_TAZ_RecreationZones_WFRC'           ,0,'Pre-Set OD'],
    ['Recreation_Zones_UtahCo'         ,'StreetLight_TAZ_RecreationZones_UtahCo'         ,0,'Pre-Set OD'],
    ['Recreation_Zones_GardenCity_Moab','StreetLight_TAZ_RecreationZones_GardenCity_Moab',0,'Pre-Set OD'],
    ['SuWs_Secondary_Homes'            ,'StreetLight_SuWs_SecondaryHomes'                ,0,'Pre-Set OD'],
    ['Recreation_Zones_Summit-Wasatch' ,'Summit Wasatch Rec'                             ,0,'Pre-Set OD'],
    ['CountyToCounty'                  ,'initialize_County_ODG_Zones'                    ,0,'OD'        ],
    ['StationAreas'                    ,'StationAreas'                                   ,0,'Pre-Set OD']
], columns=(['analysisName','zoneSetName','processZoneSet','analysisType']))
display(dfZoneSets)

# dataframe where analysis were corrupted or cancelled so they need to be renamed.
dfRedo = pd.DataFrame([
    ['Recreation_Zones_CacheCo_2019_06_TRU','Recreation_Zones_CacheCo_2019_06_TRU_2'],
    ['Recreation_Zones_CacheCo_2019_05_TRU','Recreation_Zones_CacheCo_2019_05_TRU_2'],
    ['Recreation_Zones_CacheCo_2019_04_ALL','Recreation_Zones_CacheCo_2019_04_ALL_2'],
    ['Recreation_Zones_CacheCo_2019_04_PED','Recreation_Zones_CacheCo_2019_04_PED_2'],
    ['Recreation_Zones_CacheCo_2019_04_BIC','Recreation_Zones_CacheCo_2019_04_BIC_2'],
    ['Recreation_Zones_CacheCo_2019_03_ALL','Recreation_Zones_CacheCo_2019_03_ALL_2'],
    ['Recreation_Zones_CacheCo_2019_03_BIC','Recreation_Zones_CacheCo_2019_03_BIC_2'],
    ['Recreation_Zones_CacheCo_2019_03_TRU','Recreation_Zones_CacheCo_2019_03_TRU_2']
], columns=(['analysis_set_name','analysis_set_name_new']))
display(dfRedo)

,analysisName,zoneSetName,processZoneSet,analysisType
0,Recreation_Zones_CacheCo,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD
1,Recreation_Zones_UDOT,StreetLight_TAZ_RecreationZones_UDOT,0,Pre-Set OD
2,Recreation_Zones_WasthingtonCo,StreetLight_TAZ_RecreationZones_WashingtonCo,0,Pre-Set OD
3,Recreation_Zones_WFRC,StreetLight_TAZ_RecreationZones_WFRC,0,Pre-Set OD
4,Recreation_Zones_UtahCo,StreetLight_TAZ_RecreationZones_UtahCo,0,Pre-Set OD
5,Recreation_Zones_GardenCity_Moab,StreetLight_TAZ_RecreationZones_GardenCity_Moab,0,Pre-Set OD
6,SuWs_Secondary_Homes,StreetLight_SuWs_SecondaryHomes,0,Pre-Set OD
7,Recreation_Zones_Summit-Wasatch,Summit Wasatch Rec,0,Pre-Set OD
8,CountyToCounty,initialize_County_ODG_Zones,0,OD
9,StationAreas,StationAreas,0,Pre-Set OD


,analysis_set_name,analysis_set_name_new
0,Recreation_Zones_CacheCo_2019_06_TRU,Recreation_Zones_CacheCo_2019_06_TRU_2
1,Recreation_Zones_CacheCo_2019_05_TRU,Recreation_Zones_CacheCo_2019_05_TRU_2
2,Recreation_Zones_CacheCo_2019_04_ALL,Recreation_Zones_CacheCo_2019_04_ALL_2
3,Recreation_Zones_CacheCo_2019_04_PED,Recreation_Zones_CacheCo_2019_04_PED_2
4,Recreation_Zones_CacheCo_2019_04_BIC,Recreation_Zones_CacheCo_2019_04_BIC_2
5,Recreation_Zones_CacheCo_2019_03_ALL,Recreation_Zones_CacheCo_2019_03_ALL_2
6,Recreation_Zones_CacheCo_2019_03_BIC,Recreation_Zones_CacheCo_2019_03_BIC_2
7,Recreation_Zones_CacheCo_2019_03_TRU,Recreation_Zones_CacheCo_2019_03_TRU_2


In [4]:
now = dt.datetime.utcnow().isoformat()

# Edit the following two constants to a valid key and user login.
STL_KEY = 'KnsRn7hdgOQjbywguup5hBvCoMcRMNWy'
INSIGHT_LOGIN_EMAIL = 'analytics@wfrc.org'

# create data ranges json object

dDateRanges = []

for y in range(2019, 2022): # end value not included in range
    # Create date ranges for each month of year in range
    for m in range(1, 12):
        dtMonthBeg = dt.datetime(y, m, 1)
        dtMonthEnd = dt.date(y + int(m / 12), (m % 12) + 1, 1) - dt.timedelta(days=1)
        dDateRanges.append(
            {
                'date_range_name': dtMonthBeg.strftime("%Y_%m"),
                'data_ranges'    : [{"start_date": dtMonthBeg.strftime("%m/%d/%Y") ,"end_date": dtMonthEnd.strftime("%m/%d/%Y")}]
            }
        )
        #print(dic)    
    # Add best months "BM" ranges for each year from April 1 through October 31
    dtBeg = dt.datetime(y, 4, 1)
    dtEnd = dt.datetime(y,10,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_BM",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )
    # Add best year ranges for each year
    dtBeg = dt.datetime(y, 1, 1)
    dtEnd = dt.datetime(y,12,31)
    dDateRanges.append(
        {
            'date_range_name': str(y) + "_YR",
            'data_ranges'    : [{"start_date": dtBeg.strftime("%m/%d/%Y") ,"end_date": dtEnd.strftime("%m/%d/%Y")}]
        }
    )

# for 2022 only Jan-Apr exists
for y in range(2022, 2023): # end value not included in range
    # Create date ranges for each month of year in range
    for m in range(1, 5):
        dtMonthBeg = dt.datetime(y, m, 1)
        dtMonthEnd = dt.date(y + int(m / 12), (m % 12) + 1, 1) - dt.timedelta(days=1)
        dDateRanges.append(
            {
                'date_range_name': dtMonthBeg.strftime("%Y_%m"),
                'data_ranges'    : [{"start_date": dtMonthBeg.strftime("%m/%d/%Y") ,"end_date": dtMonthEnd.strftime("%m/%d/%Y")}]
            }
        )

dfDateRanges = pd.DataFrame(dDateRanges)
#dfDateRanges.to_csv('rsg_datarequest\\streetlight_smalldist_bicycle_data\\date_ranges.csv',index=False)
#display(dfDateRanges)

# create Modes df
dModes = [
            ['TRU','Truck'       ],
            ['BIC','Bicycle'     ],
            ['PED','Pedestrian'  ],
            #['BUS','Bus'         ], # bus and rail not support in OD_Preset_Geography analysis
            #['RAI','Rail'        ],
            ['ALL','All_Vehicles']
         ]

dfModes = pd.DataFrame(dModes, columns = ['mode_name','mode_description'])
#display(dfModes)

# create cross recordset for analysis
dfAnalysisSets_ForEachZoneSet = pd.DataFrame.merge(dfDateRanges, dfModes, how='cross')
#dfAnalysisSets['analysis_set_name'] = ANALYSIS_NAME + '_' + dfAnalysisSets['date_range_name'].astype(str) + '_' + dfAnalysisSets['mode_name'].astype(str) + '_revbins'

dfAnalysisSets = pd.DataFrame()

for index, row in dfZoneSets.iterrows():
    dfAnalysisSets_ForEachZoneSet['analysis_set_name'] = row['analysisName'  ] + '_' + dfAnalysisSets_ForEachZoneSet['date_range_name'].astype(str) + '_' + dfAnalysisSets_ForEachZoneSet['mode_name'].astype(str)
    dfAnalysisSets_ForEachZoneSet['zone_set_name'    ] = row['zoneSetName'   ]
    dfAnalysisSets_ForEachZoneSet['process_zone_set' ] = row['processZoneSet']
    dfAnalysisSets_ForEachZoneSet['analysis_type'    ] = row['analysisType'  ]
    dfAnalysisSets = pd.concat([dfAnalysisSets,dfAnalysisSets_ForEachZoneSet])


# rail and bus analysis periods only include April, May, September, and October of 2019 and 2020
for index, row in dfAnalysisSets.iterrows():
    if (row['mode_description']=='Rail' or row['mode_description']=='Bus'):
        if row['date_range_name'] not in ('2019_04','2019_05','2019_09','2019_10','2020_04','2020_05','2020_09','2020_10'):
            dfAnalysisSets.drop(index, inplace=True)

# add analysis toggle
dfAnalysisSets['RunQuery'] = True

# incorporate redos
dfAnalysisSets = pd.DataFrame.merge(dfAnalysisSets,dfRedo,on='analysis_set_name', how='left')
dfAnalysisSets['analysis_set_name_new'].fillna(dfAnalysisSets['analysis_set_name'], inplace=True)
dfAnalysisSets.drop(columns=('analysis_set_name'), inplace=True)
dfAnalysisSets.rename(columns=({'analysis_set_name_new':'analysis_set_name'}), inplace=True)

# export csv
dfAnalysisSets.drop(columns=(['process_zone_set','RunQuery'])).to_csv('output\\_analysis_sets.csv',index=False)

# create download toggle
dfAnalysisSets['DataImported'] = False

display(dfAnalysisSets)

def print_response(response):
    print("response code: {}".format(response.status_code))
    print("response body: {}".format(response.content))

,date_range_name,data_ranges,mode_name,mode_description,zone_set_name,process_zone_set,analysis_type,RunQuery,analysis_set_name,DataImported
0,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",TRU,Truck,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD,True,Recreation_Zones_CacheCo_2019_01_TRU,False
1,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",BIC,Bicycle,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD,True,Recreation_Zones_CacheCo_2019_01_BIC,False
2,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",PED,Pedestrian,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD,True,Recreation_Zones_CacheCo_2019_01_PED,False
3,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",ALL,All_Vehicles,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD,True,Recreation_Zones_CacheCo_2019_01_ALL,False
4,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",TRU,Truck,StreetLight_TAZ_RecreationZones_CacheCo,0,Pre-Set OD,True,Recreation_Zones_CacheCo_2019_02_TRU,False
...,...,...,...,...,...,...,...,...,...,...
1715,2022_03,"[{'start_date': '03/01/2022', 'end_date': '03/...",ALL,All_Vehicles,StationAreas,0,Pre-Set OD,True,StationAreas_2022_03_ALL,False
1716,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",TRU,Truck,StationAreas,0,Pre-Set OD,True,StationAreas_2022_04_TRU,False
1717,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",BIC,Bicycle,StationAreas,0,Pre-Set OD,True,StationAreas_2022_04_BIC,False
1718,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",PED,Pedestrian,StationAreas,0,Pre-Set OD,True,StationAreas_2022_04_PED,False


In [28]:
dfAnalysisSets_ForEachZoneSet

,date_range_name,data_ranges,mode_name,mode_description,analysis_set_name,zone_set_name,process_zone_set,analysis_type
0,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",TRU,Truck,StationAreas_2019_01_TRU,StationAreas,0,Pre-Set OD
1,2019_01,"[{'start_date': '01/01/2019', 'end_date': '01/...",ALL,All_Vehicles,StationAreas_2019_01_ALL,StationAreas,0,Pre-Set OD
2,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",TRU,Truck,StationAreas_2019_02_TRU,StationAreas,0,Pre-Set OD
3,2019_02,"[{'start_date': '02/01/2019', 'end_date': '02/...",ALL,All_Vehicles,StationAreas_2019_02_ALL,StationAreas,0,Pre-Set OD
4,2019_03,"[{'start_date': '03/01/2019', 'end_date': '03/...",TRU,Truck,StationAreas_2019_03_TRU,StationAreas,0,Pre-Set OD
...,...,...,...,...,...,...,...,...
81,2022_02,"[{'start_date': '02/01/2022', 'end_date': '02/...",ALL,All_Vehicles,StationAreas_2022_02_ALL,StationAreas,0,Pre-Set OD
82,2022_03,"[{'start_date': '03/01/2022', 'end_date': '03/...",TRU,Truck,StationAreas_2022_03_TRU,StationAreas,0,Pre-Set OD
83,2022_03,"[{'start_date': '03/01/2022', 'end_date': '03/...",ALL,All_Vehicles,StationAreas_2022_03_ALL,StationAreas,0,Pre-Set OD
84,2022_04,"[{'start_date': '04/01/2022', 'end_date': '04/...",TRU,Truck,StationAreas_2022_04_TRU,StationAreas,0,Pre-Set OD


In [29]:
dfAnalysisSets[dfAnalysisSets['analysis_set_name'].str.contains('WFRC_2019_YR_ALL')]

,date_range_name,data_ranges,mode_name,mode_description,zone_set_name,process_zone_set,analysis_type,RunQuery,analysis_set_name,DataImported
283,2019_YR,"[{'start_date': '01/01/2019', 'end_date': '12/...",ALL,All_Vehicles,StreetLight_TAZ_RecreationZones_WFRC,0,Pre-Set OD,True,Recreation_Zones_WFRC_2019_YR_ALL,False


In [30]:
#----------------------------------------------------------------------------------------
# Create a Zone Set.
#----------------------------------------------------------------------------------------

#ZONE_SET_REQUEST = {
#    "insight_login_email": INSIGHT_LOGIN_EMAIL,
#    "zone_set_name": ZONE_SET_NAME,
#    "zones": {
#        "type": "FeatureCollection",
#        "features": [
#            { "type": "Feature", "properties": { "id": 1, "name": "Mission", "is_pass": 0, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.426698258661972, 37.769562689936315 ], [ -122.423394859041892, 37.772083876030344 ], [ -122.42225575572462, 37.770418101996292 ], [ -122.411206453547067, 37.769427623969634 ], [ -122.406991771273155, 37.769067446852375 ], [ -122.404656609472752, 37.767716767038294 ], [ -122.405169205965521, 37.762628984940662 ], [ -122.406593085112107, 37.760557752167074 ], [ -122.405795712790038, 37.758441432691242 ], [ -122.403232730326167, 37.75722564731312 ], [ -122.402549268335804, 37.751821915004783 ], [ -122.403346640657901, 37.749390106706535 ], [ -122.407561322931784, 37.748399347080543 ], [ -122.424875693354338, 37.74781389197571 ], [ -122.426698258661972, 37.769562689936315 ] ] ] ] } },
#            { "type": "Feature", "properties": { "id": 2, "name": "Financial District", "is_pass": 1, "direction": None }, "geometry": { "type": "MultiPolygon", "coordinates": [ [ [ [ -122.405425504211919, 37.798033588378779 ], [ -122.398476973976571, 37.798933675051543 ], [ -122.396654408668923, 37.799698740100226 ], [ -122.397024617247041, 37.79929370664977 ], [ -122.396768319000643, 37.798911173018389 ], [ -122.395828558763895, 37.797898574431919 ], [ -122.393607307295227, 37.799113691070012 ], [ -122.392610591892605, 37.797876072083447 ], [ -122.394233814119715, 37.79690846455324 ], [ -122.393037755636598, 37.795670808599994 ], [ -122.3913575782436, 37.796278387822021 ], [ -122.390987369665496, 37.795693311702443 ], [ -122.392439726395011, 37.794500642185817 ], [ -122.389278714689581, 37.791462623416393 ], [ -122.401182344355107, 37.781965196242638 ], [ -122.405824190372982, 37.785701296639232 ], [ -122.406222876534017, 37.785723802695827 ], [ -122.407134159187834, 37.790337399578668 ], [ -122.404058580231194, 37.790764986655553 ], [ -122.405425504211919, 37.798033588378779 ] ] ] ] } }
#        ]
#    }
#}
#
#resp = requests.post(
#    'https://insight.streetlightdata.com/api/v2/zone_sets',
#    headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
#    data = json.dumps(ZONE_SET_REQUEST))
#
#print_response(resp)
#
#if (resp.status_code == 201):
#    print("Created Zone Set successfully.")
#else:
#    print("Error creating Zone Set.")
#    sys.exit(1)
#
#


In [66]:
#----------------------------------------------------------------------------------------
# Create an O-D Analysis.
#----------------------------------------------------------------------------------------

for index, row in dfAnalysisSets.iterrows():

    if row['process_zone_set'] and index>=763:

        if row['RunQuery']==True:

            print(row['analysis_set_name'] + ' index: ' + str(index))

            _dayparts = "All Day|0023, HR00|0000, HR01|0101, HR02|0202, HR03|0303, HR04|0404, HR05|0505, HR06|0606, HR07|0707, HR08|0808, HR09|0909, HR10|1010, HR11|1111, HR12|1212, HR13|1313, HR14|1414, HR15|1515, HR16|1616, HR17|1717, HR18|1818, HR19|1919, HR20|2020, HR21|2121, HR22|2222, HR23|2323"
            _daytypes = "All Days|17, Mo|11, Tu|22, We|33, Th|44, Fr|55, Sa|66, Su|77, Weekday_TuWeTh|24, Weekend_SaSu|67"

            if row['analysis_type']=='Pre-Set OD':
                CREATE_ANALYSIS_REQUEST = {
                    "insight_login_email": INSIGHT_LOGIN_EMAIL,
                    "analysis_name": row['analysis_set_name'],
                    "analysis_type": "OD_Preset_Geography",
                    "travel_mode_type": row['mode_description'],
                    "description": "",
                    "oz_sets": [{"name":row['zone_set_name']}],
                    "geography_type": "blkgrp",
                    "date_ranges": row['data_ranges'],
                    "day_types": _daytypes,
                    "day_parts": _dayparts,
                    "trip_attributes": True
                }
            elif row['analysis_type']=='OD':
                CREATE_ANALYSIS_REQUEST = {
                    "insight_login_email": INSIGHT_LOGIN_EMAIL,
                    "analysis_name": row['analysis_set_name'],
                    "analysis_type": "OD_Analysis",
                    "travel_mode_type": row['mode_description'],
                    "description": "",
                    "oz_sets": [{"name":row['zone_set_name']}],
                    "dz_sets": [{"name":row['zone_set_name']}],
                    "date_ranges": row['data_ranges'],
                    "day_types": _daytypes,
                    "day_parts": _dayparts,
                    "trip_attributes": True
                }

            # traveler attributes only exist for non-Truck modes
            if (row['mode_description']!='Truck'):
                CREATE_ANALYSIS_REQUEST['traveler_attributes'] = True
                #CREATE_ANALYSIS_REQUEST['truck_weight'] = 'Medium,Heavy'

            #print(CREATE_ANALYSIS_REQUEST)
            resp = requests.post(
                'https://insight.streetlightdata.com/api/v2/analyses',
                headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
                data = json.dumps(CREATE_ANALYSIS_REQUEST))

            print_response(resp)

            if (resp.status_code == 201):
                print("Created Analysis successfully.")
                dfAnalysisSets.loc[index,'RunQuery'] = False
                time.sleep(10) # wait ten seconds since URL has max limit of 3 calls per 15 seconds
            else:
                print("Error creating Analysis.")
                sys.exit(1)


CountyToCounty_2022_01_ALL index: 767
response code: 201
response body: b'{"name":"CountyToCounty_2022_01_ALL","status":"success","uuid":"7adec2ce-2ab0-4bb3-a662-b41e66d8c41b"}\n'
Created Analysis successfully.
CountyToCounty_2022_02_TRU index: 768
response code: 201
response body: b'{"name":"CountyToCounty_2022_02_TRU","status":"success","uuid":"3bf4954b-8bc9-4935-b412-5570c6e66759"}\n'
Created Analysis successfully.
CountyToCounty_2022_02_ALL index: 769
response code: 201
response body: b'{"name":"CountyToCounty_2022_02_ALL","status":"success","uuid":"6928892c-69ce-4e71-b394-1f9270e2778b"}\n'
Created Analysis successfully.
CountyToCounty_2022_03_TRU index: 770
response code: 201
response body: b'{"name":"CountyToCounty_2022_03_TRU","status":"success","uuid":"268c189f-414d-40b9-8bb6-4ee180b27417"}\n'
Created Analysis successfully.
CountyToCounty_2022_03_ALL index: 771
response code: 201
response body: b'{"name":"CountyToCounty_2022_03_ALL","status":"success","uuid":"d12ef447-5d35-4445

In [16]:
#----------------------------------------------------------------------------------------
# Check the processing status of the Analysis.
#
# Note: depending on Analysis size, Analysis processing can take minutes to hours
# (for very large Analyses). A production integration should not block UI input
# from the end user while Analyses are processing, and it should resume
# gracefully when the client application is shut down and restarted while
# Analyses process.
#----------------------------------------------------------------------------------------

if False:
    #pick a month that has all modes
    dfOneOfEachMode = dfAnalysisSets[dfAnalysisSets['analysis_set_name'].str.contains('2022_01')]


    for index, row in dfOneOfEachMode.iterrows():

        print(row['analysis_set_name'])

        CHECK_STATUS_REQUEST = {
            "analyses":[{"name": row['analysis_set_name']}]
        }

        resp = requests.post(
            'https://insight.streetlightdata.com/api/v2/analyses/status',
            headers = {'content-type': 'application/json', 'x-stl-key': STL_KEY},
            data = json.dumps(CHECK_STATUS_REQUEST))

        print_response(resp)

        if (resp.status_code != 200):
            print("Error checking Analysis Status.")
            sys.exit(1)

        json_result = json.loads(resp.text)
        analysis_status = json_result["analyses"][0]["status"]

        if (analysis_status == "Available"):
            print("Analysis is Available!")
            time.sleep(10)
            break
        elif (analysis_status == "Processing"):
            print("Analysis is processing. Trying again after 1 minute...")
            time.sleep(10)
        else:
            print(analysis_status)
            time.sleep(10)
            #print("Error running Analysis.")
            #sys.exit(1)
    #    
    #    # only go through first time to get available downloades
    #    break

In [6]:
dMetricInputDic = [
    ['TRU','odg_comm',['Vehicle Weight','Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Index)' ,'Avg Travel Time (sec)'], ['truck_weight','OName','DName','day_type','day_part','odvolume','odtime']],
   #['BUS','odg_bus' ,[                 'Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Index)' ,'Avg Travel Time (sec)'], [               'OName','DName','day_type','day_part','odvolume','odtime']],
    ['BIC','odg_bike',[                 'Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Volume)','Avg Travel Time (sec)'], [               'OName','DName','day_type','day_part','odvolume','odtime']],
    ['PED','odg_ped' ,[                 'Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Volume)','Avg Travel Time (sec)'], [               'OName','DName','day_type','day_part','odvolume','odtime']],
   #['RAI','odg_rail',[                 'Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Index)' ,'Avg Travel Time (sec)'], [               'OName','DName','day_type','day_part','odvolume','odtime']],
    ['ALL','odg_all' ,[                 'Origin Zone Name','Destination Zone Name','Day Type','Day Part','Average Daily O-D Traffic (StL Index)' ,'Avg Travel Time (sec)'], [               'OName','DName','day_type','day_part','odvolume','odtime']]
]
dfMetricInputDic = pd.DataFrame(dMetricInputDic, columns = ['mode_name','metric_name','keep_fields','keep_fields_new'])
display(dfMetricInputDic)

#----------------------------------------------------------------------------------------
# Get the O-D results.
#----------------------------------------------------------------------------------------

#"analyses":[{"metrics":["od_comm","zone_od_comm","od_trip_comm","zone_trip_comm"]

dfImport = pd.DataFrame()

for index, row in dfAnalysisSets.iterrows():

    if row['DataImported']==False:

        print("Importing " + row['analysis_set_name'] + " (index: " + str(index) +  ")...")

        _metric_name = dfMetricInputDic.loc[(dfMetricInputDic['mode_name']==row['mode_name']), 'metric_name'].values[0]
        
        resp = requests.get(
            'https://insight.streetlightdata.com/api/v2/analyses/download/name/{}'.format(row['analysis_set_name']) + '/' + _metric_name,
            headers = {'x-stl-key': STL_KEY})

        # resp.text contains results in CSV format.
        #print_response(resp)

        if (resp.status_code == 200):
            # Write results to a CSV file.        
            with open("temp.csv", "w") as csv_file:
                csv_file.write(resp.text)
            
            # read csv into temporary dataframe
            dfTemp = pd.read_csv("temp.csv",usecols=dfMetricInputDic.loc[(dfMetricInputDic['mode_name']==row['mode_name']), 'keep_fields'].values[0])
            
            #display(dfTemp)

            # rename columns to make shorter
            dfTemp.columns = dfMetricInputDic.loc[(dfMetricInputDic['mode_name']==row['mode_name']), 'keep_fields_new'].values[0]
            #display(dfTemp)

            # add columns with data from analysis set fields
            #dfTemp['mode'      ] = row['mode_name'      ]
            #dfTemp['date_range'] = row['date_range_name']

            # append temporary dataframe to Import dataframe
            #dfImport = pd.concat([dfImport, dfTemp])
            dfImport = dfTemp
            
            dfImport['odtime'] = dfImport['odtime'].fillna(0).astype(int)

            dfImport['OName'] = dfImport['OName'].str.replace("'","")
            dfImport['DName'] = dfImport['DName'].str.replace("'","")
            
            dfImport['day_type_code'] = dfImport['day_type'].str.split(':', n=1).str.get(0)
            dfImport['day_part_code'] = dfImport['day_part'].str.split(':', n=1).str.get(0)

            #display(dfImport)
            
#            if index==567: # pick a WFRC all year row that is sure to have all the type and part codes
#                dfImport[['day_type_code','day_type']].drop_duplicates().to_csv('output\\_day_type_codes.csv',index=False)
#                dfImport[['day_part_code','day_part']].drop_duplicates().to_csv('output\\_day_part_codes.csv',index=False)
            
            dfImport.drop(columns=(['day_type','day_part'])).to_csv('output\\results-by-analysis\\' +row['analysis_set_name'] + '.csv',index=False)
            
            dfAnalysisSets.loc[index,'DataImported'] = True

            # pause for 2 seconds due to API restrictions
            time.sleep(2)
            
            print("Done.")
        else:
            print("Error fetching O-D results.")
            sys.exit(1)

,mode_name,metric_name,keep_fields,keep_fields_new
0,TRU,odg_comm,"[Vehicle Weight, Origin Zone Name, Destination...","[truck_weight, OName, DName, day_type, day_par..."
1,BIC,odg_bike,"[Origin Zone Name, Destination Zone Name, Day ...","[OName, DName, day_type, day_part, odvolume, o..."
2,PED,odg_ped,"[Origin Zone Name, Destination Zone Name, Day ...","[OName, DName, day_type, day_part, odvolume, o..."
3,ALL,odg_all,"[Origin Zone Name, Destination Zone Name, Day ...","[OName, DName, day_type, day_part, odvolume, o..."


Importing Recreation_Zones_Summit-Wasatch_2019_01_TRU (index: 1204)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_01_BIC (index: 1205)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_01_PED (index: 1206)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_01_ALL (index: 1207)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_02_TRU (index: 1208)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_02_BIC (index: 1209)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_02_PED (index: 1210)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_02_ALL (index: 1211)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_03_TRU (index: 1212)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_03_BIC (index: 1213)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_03_PED (index: 1214)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_03_ALL (index: 1215)...
Done.
Importing Recreation_Zones_Summit-Wasatch_2019_04_TRU (index: 1216)...
Done.

In [ ]:
dfImport[['day_type_code','day_type']].drop_duplicates().to_csv('day_type_codes.csv',index=False)

In [ ]:
dfImport

,truck_weight,OName,DName,day_type,day_part,odvolume,odtime,mode,date_range,day_type_code,day_part_code
0,Medium,50880_2,490050001011,0: All Days (M-Su),00: All Day (12am-12am),16,4490,TRU,2019_01,0,00
1,Medium,50880_2,490050001011,0: All Days (M-Su),16: HR15 (3pm-4pm),5,3662,TRU,2019_01,0,16
2,Medium,50880_2,490050001011,0: All Days (M-Su),22: HR21 (9pm-10pm),11,4904,TRU,2019_01,0,22
3,Medium,50880_2,490050001011,1: Mo (M-M),00: All Day (12am-12am),41,3662,TRU,2019_01,1,00
4,Medium,50880_2,490050001011,1: Mo (M-M),16: HR15 (3pm-4pm),41,3662,TRU,2019_01,1,16
...,...,...,...,...,...,...,...,...,...,...,...
149,Heavy,490490101323,50888_2,0: All Days (M-Su),13: HR12 (12noon-1pm),11,8430,TRU,2019_01,0,13
150,Heavy,490490101323,50888_2,2: Tu (Tu-Tu),00: All Day (12am-12am),65,8430,TRU,2019_01,2,00
151,Heavy,490490101323,50888_2,2: Tu (Tu-Tu),13: HR12 (12noon-1pm),65,8430,TRU,2019_01,2,13
152,Heavy,490490101323,50888_2,3: Weekday_TuWeTh (Tu-Th),00: All Day (12am-12am),22,8430,TRU,2019_01,3,00
